## Cost Update

Test methods for updating the global costmap from local observations

In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import cv2 as cv
# import GPy
from scipy.interpolate import RBFInterpolator
from sklearn.neighbors import KernelDensity

from nerfnav.global_planner import GlobalPlanner
from nerfnav.utils import rgb2gray, pc_plot_trace
from nerfnav.feature_map import FeatureMap, CostMap
from nerfnav.autonav import AutoNav

%load_ext autoreload
%autoreload 2

### Parameters and Setup

In [ ]:
UNREAL_PLAYER_START = np.array([-117252.054688, 264463.03125, 25148.908203])
UNREAL_GOAL = np.array([-83250.0, 258070.0, 24860.0])

GOAL_POS = (UNREAL_GOAL - UNREAL_PLAYER_START)[:2] / 100.0

global_img = cv.imread('../data/airsim/images/test_scenario.png')
global_img = global_img[::2, ::2, :]
start_px = (138, 141)
goal_px = (78, 493)

### Global costmap and planner

In [ ]:
costmap_data = np.load('../data/airsim/costmap.npz')
costmap = CostMap(costmap_data['mat'], costmap_data['cluster_labels'], costmap_data['cluster_masks'])

feat_map = FeatureMap(global_img, start_px, goal_px, UNREAL_PLAYER_START, UNREAL_GOAL)
global_planner = GlobalPlanner(costmap, feat_map, goal_px)

#init_path = global_planner.replan(np.zeros(3))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(global_img)
im = global_planner.plot(ax[1])
# plt.colorbar(im, ax=ax[1])
plt.show()

### Local observation

In [ ]:
depth = np.load('../data/airsim/images/rover/depth_start.npy')
#pose = np.array([169.813, 23.864, -0.104])  # x, y, yaw
pose = np.array([0.0, 0.0, 0.0])

depth_vis = depth.copy()
depth_vis[depth_vis > 100] = 100.0

autonav = AutoNav(np.zeros(2))
cost_vals = autonav.update_costmap(pose, depth)
fig, ax = plt.subplots(1, 2, figsize=(15, 5))
ax[0].imshow(depth_vis)
im = autonav.plot_costmap(ax[1])
plt.colorbar(im, fraction=0.04, aspect=12)

In [ ]:
cvs = np.array(cost_vals)
fig, ax = plt.subplots(1, 1, figsize=(100, 50))
s = ax.scatter(cvs[:, 0], cvs[:, 1], c=cvs[:, 2], cmap='viridis_r', s=10)
ax.imshow(global_img, extent=feat_map.bounds)
#ax.invert_yaxis()
plt.axis('equal')
plt.show()

In [ ]:
# Plot cvs in 3D with plotly
data = pc_plot_trace(cvs, color='blue')
fig = go.Figure(data=data)
fig.show()

In [ ]:
cost_vals[:,0]

In [ ]:
i, j = feat_map.get_img_coords(np.zeros((10, 2)))

In [ ]:
global_planner.update_costmap(cost_vals)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 10))
ax[0].imshow(global_img)
im = global_planner.plot(ax[1])
# plt.colorbar(im, ax=ax[1])
plt.show()

### Cost interpolation

In [ ]:
c_idx = 1

X, Y = feat_map.img_to_global(costmap.cluster_idxs[c_idx][:,0], costmap.cluster_idxs[c_idx][:,1])

coords = costmap.cluster_idxs[c_idx]
RGB = global_img[coords[:,0], coords[:,1], :]

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.scatter(X, Y, s=10, c=RGB/255.0)
ax.scatter(cvs[:, 0], cvs[:, 1], c=cvs[:, 2], cmap='viridis_r', s=5)
plt.axis('equal')
plt.show()

In [ ]:
x = costmap.cluster_idxs[c_idx][:,0]
y = costmap.cluster_idxs[c_idx][:,1]
costmap.mat[x, y].shape

In [ ]:
costmap.cluster_masks[c_idx]

In [ ]:
sample_RGB = feat_map.get_features(cvs[:,:2])

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.scatter(cvs[:, 0], cvs[:, 1], c=sample_RGB/255.0, s=70)
plt.axis('equal')

In [ ]:
norm_const = max(np.abs([min(X), max(X), min(Y), max(Y)]))
norm_const = 200

In [ ]:
features = np.hstack((X[:,None]/norm_const, Y[:,None]/norm_const, RGB/255.0))

In [ ]:
sample_features = np.hstack((cvs[:, :2]/norm_const, sample_RGB/255.0))

In [ ]:
# Spatial + feature
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sample_features, cvs[:, 2])
density = np.exp(kde.score_samples(features))

# Feature only
# kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(sample_RGB/255.0, cvs[:, 2])
# density = np.exp(kde.score_samples(RGB/255.0))

In [ ]:
coords = costmap.cluster_idxs[c_idx]
RGB = global_img[coords[:,0], coords[:,1], :]

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
im = ax.scatter(X, Y, s=10, c=density, cmap='viridis')
plt.colorbar(im, fraction=0.04, aspect=12)
#ax.scatter(cvs[:, 0], cvs[:, 1], c=cvs[:, 2], cmap='viridis_r', s=5)
plt.axis('equal')
plt.show()

In [ ]:
from nerfnav.utils import fibonacci_sphere

fibonacci_sphere(100)